# Gallery Example: Erl/Erl/1 Reentrant Queue

This example demonstrates an Erlang/Erlang/1 queueing system with reentrant routing:
- **Arrivals**: Erlang distribution
- **Service**: Erlang distribution (Class1), Exponential (Class2)
- **Servers**: 1 server
- **Scheduling**: FCFS
- **Reentrant**: Jobs may switch class and pass through the queue again

This model includes probabilistic class switching.

In [ ]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_erlerl1_reentrant(n=5):
    """Create Erl/Erl/1-Reentrant queueing model"""
    model = Network('Erl/Erl/1-Reentrant')
    
    # Block 1: nodes
    source = Source(model, 'Source')
    queue = Queue(model, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model, 'Sink')
    
    # Block 2: classes
    oclass1 = OpenClass(model, 'Class1')
    oclass2 = OpenClass(model, 'Class2')
    
    # Erlang arrival for Class1, Class2 is disabled at source
    source.set_arrival(oclass1, Erlang.fit_mean_and_order(1, n))
    source.set_arrival(oclass2, Disabled())
    
    # Erlang service for Class1, Exponential for Class2
    queue.set_service(oclass1, Erlang.fit_mean_and_order(0.5, n))
    queue.set_service(oclass2, Exp(3))
    
    # Block 3: topology with probabilistic class switching
    P = model.init_routing_matrix()
    P.add_route(oclass1, source, queue, 1.0)        # Arrivals go to queue as Class1
    P.add_class_switch(oclass1, oclass2, queue, queue, 0.5)  # 50% switch to Class2
    P.add_route(oclass2, queue, sink, 0.5)          # 50% of Class2 exits
    model.link(P)
    
    return model

# Create the model
model = gallery_erlerl1_reentrant()

## About This Model

This reentrant model features:
- Both Erlang arrivals and Erlang service (low variability)
- Probabilistic class switching: 50% chance to switch from Class1 to Class2
- Different service distributions by class

The low variability of Erlang distributions (SCV = 1/n) combined with reentrant routing creates interesting dynamics in the queue.

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)